In [1]:
from pathlib import Path
import json
from configparser import ConfigParser

import requests
import psycopg
import pandas as pd
import geopandas as gpd


In [2]:
parser = ConfigParser()
# read config file
parser.read("database.ini")
params = dict(parser["postgresql"])


# Set up database

Schema: 

![Schema](../data/Schema.png)

In [8]:
with psycopg.connect(**params) as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:

        # Execute a command: this creates a new table
        cur.execute("""
            CREATE TABLE Countries (
                ISO text PRIMARY KEY,
                EnglishName text,
                Endonym text,
                ADM0simple geometry)
            """)
        cur.execute("""
            CREATE TABLE ADM1 (
                ADM1_ID SERIAL PRIMARY KEY NOT NULL,
                Country_ISO text,
                Name text,
                ADM1simple geometry,
                FOREIGN KEY (Country_ISO) REFERENCES Countries(ISO))
            """)
        cur.execute("""
            CREATE TABLE Organizations (
                Org_ID text PRIMARY KEY,
                OrgName text
            );
            """)
        cur.execute("""
            CREATE TABLE Memberships (
                Country_ISO text,
                Org_ID text,
                PRIMARY KEY (Country_ISO, Org_ID),
                FOREIGN KEY (Country_ISO) REFERENCES Countries(ISO),
                FOREIGN KEY (Org_ID) REFERENCES Organizations(Org_ID)
            );
            """)

## Initialize International Organizations and country memberships

Repo as basis: https://github.com/dieghernan/Country-Codes-and-International-Organizations/blob/master/outputs/CountrycodesOrgs.csv

This writes a json in the format `{"SHORT_CODE_OF_ORG": {org_name: "FULL ORG NAME", members: [GER, DEN, BEL, etc..]}}`

In [9]:
organizations = pd.read_csv("../data/CountrycodesOrgs.csv")
orgs = organizations[organizations.org_member == "member"].groupby("org_id")

In [10]:
org_dicts = {}

for name, group in orgs:
    org_dicts.update({name: dict(
        org_name = group.org_name.values[0].strip(),
        members = list(group.ISO_3166_3.values)
    )})

In [11]:
with open("../data/orgs.json", "w+") as fs:
    json.dump(org_dicts, fs, indent=2)

In [74]:
with open("../data/orgs.json", "r") as fs:
    org_dicts = json.load(fs)

with psycopg.connect(**params, autocommit=True) as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:
        for org_short, values in org_dicts.items():
            cur.execute("INSERT INTO Organizations (Org_ID, OrgName) VALUES (%s, %s)",
                        (org_short, values["org_name"]))
            for member in values["members"]:
                try:
                    cur.execute("INSERT INTO Memberships (Org_ID, Country_ISO) VALUES (%s, %s)",
                            (org_short, member))
                except psycopg.errors.ForeignKeyViolation:
                    print(f"{member} in Org {values['org_name']} not a country in Countries table.")
                    conn.rollback()

LBN in Org Arab Bank for Economic Development in Africa not a country in Countries table.
HKG in Org Asian Development Bank not a country in Countries table.
LBN in Org Arab Fund for Economic and Social Development not a country in Countries table.
LBN in Org Arab League not a country in Countries table.
LBN in Org Arab Monetary Fund not a country in Countries table.
HKG in Org Asia-Pacific Economic Cooperation not a country in Countries table.
ESH in Org African Union not a country in Countries table.
HKG in Org Bank for International Settlements not a country in Countries table.
LBN in Org Council of Arab Economic Unity not a country in Countries table.
LBN in Org Food and Agriculture Organization not a country in Countries table.
HKG in Org Financial Action Task Force not a country in Countries table.
LBN in Org Group of 24 not a country in Countries table.
LBN in Org Group of 77 not a country in Countries table.
LBN in Org International Atomic Energy Agency not a country in Countri

## Initialize country table

This intitializes the country table. 

In [30]:
all_adm0 = requests.get("https://www.geoboundaries.org/api/current/gbOpen/ALL/ADM0/").json()
iso_list = [country["boundaryISO"] for country in all_adm0]

In [31]:
def border_generator(adm, isos):
    base_url = "https://www.geoboundaries.org/api/current/gbOpen/"
    for iso in isos:
        api_url = f"{base_url}/{iso}/{adm}"
        country = requests.get(api_url)
        try:
            country.raise_for_status()
        except requests.HTTPError:
            if country.status_code == 404:
                continue
            
        country_meta = country.json()
        country_geojson = requests.get(country_meta["simplifiedGeometryGeoJSON"])
        geoms = gpd.read_file(country_geojson.text)
        for geom in geoms.itertuples():
            yield iso, geom.shapeName, geom.geometry.wkb_hex
            

The next cell adds the geometries for ADM0 to the database. Some might fail due to no file available

In [32]:
with psycopg.connect(autocommit=True, **params) as conn:
    with conn.cursor() as cur:
        for iso, countryName, geom in border_generator("ADM0", iso_list):
            print(iso)
            cur.execute("INSERT INTO Countries (ISO, EnglishName, ADM0Simple) VALUES (%s, %s, ST_SetSRID(%s::geometry, %s))",
                        (iso, countryName, geom, 4326))

ABW
AFG
AGO
AIA
ALB
AND
ARE
ARG
ARM
ASM
ATA
ATG
AUS
AUT
AZE
BDI
BEL
BEN
BES
BFA
BGD
BGR
BHR
BHS
BIH
BLM
BLR
BLZ
BMU
BOL
BRA
BRB
BRN
BTN
BWA
CAF
CAN
CHE
CHL
CHN
CIV
CMR
COD
COG
COK
COL
COM
CPV
CRI
CUB
CUW
CYM
CYP
CZE
DEU
DJI
DMA
DNK
DOM
DZA
ECU
EGY
ERI
ESP
EST
ETH
FIN
FJI
FLK
FRA
FRO
FSM
GAB
GBR
GEO
GGY
GHA
GIB
GIN
GLP
GMB
GNB
GNQ
GRC
GRD
GRL
GTM
GUF
GUM
GUY
HND
HRV
HTI
HUN
IDN
IMN
IND
IRL
IRN
IRQ
ISL
ISR
ITA
JAM
JOR
JPN
KAZ
KEN
KGZ
KHM
KIR
KNA
KOR
KWT
LAO
LBR
LBY
LCA
LIE
LKA
LSO
LTU
LUX
LVA
MAR
MCO
MDA
MDG
MDV
MEX
MHL
MKD
MLI
MLT
MMR
MNE
MNG
MNP
MOZ
MRT
MSR
MTQ
MUS
MWI
MYS
MYT
NAM
NCL
NER
NGA
NIC
NIU
NLD
NOR
NPL
NRU
NZL
OMN
PAK
PAN
PCN
PER
PHL
PLW
PNG
POL
PRK
PRT
PRY
PSE
PYF
QAT
REU
ROU
RUS
RWA
SAU
SDN
SEN
SGP
SHN
SLB
SLE
SLV
SMR
SOM
SRB
SSD
STP
SUR
SVK
SVN
SWE
SWZ
SYC
SYR
TCA
TCD
TGO
THA
TJK
TKM
TLS
TON
TTO
TUN
TUR
TUV
TWN
TZA
UGA
UKR
URY
USA
UZB
VAT
VCT
VEN
VGB
VIR
VNM
VUT
WLF
WSM
XKX
YEM
ZAF
ZMB
ZWE


## Add in the ADM1 geojsons

In [37]:
with psycopg.connect(autocommit=True, **params) as conn:
    with conn.cursor() as cur:
        for iso, countryName, geom in border_generator("ADM1", iso_list):
            print(countryName)
            cur.execute("INSERT INTO ADM1 (Country_ISO, Name, ADM1Simple) VALUES (%s, %s, ST_SetSRID(%s::geometry, %s))",
                        (iso, countryName, geom, 4326))

Kandahar
Zabul
Uruzgan
Daykundi
Ghanzi
Paktika
Khost
Paktia
Logar
Wardak
Kabul
Nangarhar
Laghman
Kapisa
Parwan
Panjshir
Kunar
Nuristan
Baghlan
Bamyan
Samangan
Kunduz
Takhar
Balkh
Sar-e Pol
Jowzjan
Faryab
Badghis
Ghor
Herat
Farah
Nimruz
Helmand
Badakhshan
Bengo
Benguela
Bié
Cabinda
Cunene
Huambo
Huíla
Cuando Cubango
Cuanza Norte
Cuanza Sul
Luanda
Lunda Norte
Lunda Sul
Malanje
Moxico
Namibe
Uíge
Zaire
Berat
Dibër
Durrës
Elbasan
Fier
Gjirokastër
Korçë
Kukës
Lezhë
Shkodër
Tiranë
Vlorë
Sant Julia de Loria
Canillo
Ordino
La Massana
Encamp
Escaldes-Engordany
Andorra la Vella
Abu Dhabi
Ajman
Dubai
Fujairah
Ras al-Khaimah
Sharjah
Umm al-Quwain
Buenos Aires
Catamarca
Chaco
Chubut
Ciudad Autónoma de Buenos Aires
Córdoba
Corrientes
Formosa
Jujuy
La Pampa
La Roja
Mendoza
Misiones
Neuquén
Río Negro
Salta
San Juan
San Luis
Santa Cruz
Santa Fe
Santiago del Estero
Tierra del Fuego
Tucumán
Gegharkunik
Syunik
Tavush
Shirak
Lori
Kotayk
Vayots Dzor
Armavir
Aragatsotn
Yerevan
Ararat
Redonda
Saint Philip
Sai

## Query the database and return GeoJSONs

That next boy gets a single row of the table and returns its geometry as a geojson. Sweet as.

In [38]:
def get_country_geojson(iso):
    with psycopg.connect(**params) as conn:
        with conn.cursor() as cur:
            cur.execute(
                """
    SELECT jsonb_build_object(
    'type',       'Feature',
    'id',         iso,
    'geometry',   ST_AsGeoJSON(ADM0simple)::jsonb,
    'properties', to_jsonb(row) - 'iso' - 'simplegeom'
    ) FROM (SELECT * FROM countries WHERE iso=%s) row LIMIT 1;""",
    [iso]
            )
            return cur.fetchone()[0]

In [23]:
def get_adm1_geojson(adm1_id):
    with psycopg.connect(**params) as conn:
        with conn.cursor() as cur:
            cur.execute(
                """
    SELECT jsonb_build_object(
    'type',       'Feature',
    'id',         name,
    'geometry',   ST_AsGeoJSON(ADM1simple)::jsonb,
    'properties', to_jsonb(row) - 'iso' - 'simplegeom'
    ) FROM (SELECT * FROM ADM1 WHERE ADM1_ID={adm1_id}) row LIMIT 1;"""
            )
            return cur.fetchone()[0]

In [39]:
test = get_country_geojson("BEL")
test

{'id': 'BEL',
 'type': 'Feature',
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[5.10218, 51.429005],
     [5.081105, 51.466647],
     [5.049276, 51.472331],
     [5.03382, 51.484649],
     [5.014763, 51.474288],
     [5.000885, 51.44337],
     [4.92708, 51.396821],
     [4.884981, 51.414596],
     [4.785251, 51.411103],
     [4.773403, 51.416937],
     [4.772836, 51.428643],
     [4.826744, 51.425198],
     [4.825754, 51.446873],
     [4.83904, 51.478104],
     [4.806476, 51.49684],
     [4.759926, 51.502464],
     [4.750611, 51.497989],
     [4.746145, 51.489613],
     [4.728776, 51.483376],
     [4.715785, 51.469155],
     [4.704348, 51.466608],
     [4.692802, 51.452317],
     [4.667286, 51.444439],
     [4.669544, 51.426384],
     [4.577543, 51.431913],
     [4.53833, 51.424937],
     [4.531711, 51.445505],
     [4.542871, 51.476821],
     [4.478998, 51.477104],
     [4.392661, 51.450081],
     [4.396215, 51.432564],
     [4.387892, 51.415014],
     [4.427319, 51.37715

In [72]:
with open("two.geojson", "w") as fs:
    json.dump(get_countries_geojson(["BEL", "DEU"]), fs)

In [26]:
with open("durres.geojson", "w") as fs:
    json.dump(test, fs)

In [16]:
with psycopg.connect(**params) as conn:
    with conn.cursor() as cur:
        cur.execute(
            """
SELECT * FROM Countries WHERE iso='{iso}'"""
        )
        for thing in cur:
            print(thing)


In [70]:
def get_countries_geojson(isos):
    with psycopg.connect(**params) as conn:
        with conn.cursor() as cur:
            cur.execute("""
SELECT jsonb_build_object(
    'type', 'FeatureCollection',
    'features', jsonb_agg(features.feature)
)
FROM (
    SELECT jsonb_build_object(
        'type', 'Feature',
        'id', iso,
        'geometry', ST_AsGeoJSON(ADM0simple)::jsonb,
        'properties', to_jsonb(row) - 'iso' - 'simplegeom'
    ) AS feature
    FROM (
        SELECT *
        FROM countries
        WHERE iso = ANY(%s)
    ) row
) features;
""",
[isos]
            )
            return cur.fetchone()[0]


In [71]:
get_countries_geojson(["BEL", "DEU"])

{'type': 'FeatureCollection',
 'features': [{'id': 'BEL',
   'type': 'Feature',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[5.10218, 51.429005],
       [5.081105, 51.466647],
       [5.049276, 51.472331],
       [5.03382, 51.484649],
       [5.014763, 51.474288],
       [5.000885, 51.44337],
       [4.92708, 51.396821],
       [4.884981, 51.414596],
       [4.785251, 51.411103],
       [4.773403, 51.416937],
       [4.772836, 51.428643],
       [4.826744, 51.425198],
       [4.825754, 51.446873],
       [4.83904, 51.478104],
       [4.806476, 51.49684],
       [4.759926, 51.502464],
       [4.750611, 51.497989],
       [4.746145, 51.489613],
       [4.728776, 51.483376],
       [4.715785, 51.469155],
       [4.704348, 51.466608],
       [4.692802, 51.452317],
       [4.667286, 51.444439],
       [4.669544, 51.426384],
       [4.577543, 51.431913],
       [4.53833, 51.424937],
       [4.531711, 51.445505],
       [4.542871, 51.476821],
       [4.478998, 51.477104],
  

In [77]:
def get_org_geojson(orgname):
    with psycopg.connect(**params) as conn:
        with conn.cursor() as cur:
            cur.execute("""
SELECT jsonb_build_object(
    'type', 'FeatureCollection',
    'features', jsonb_agg(features.feature)
)
FROM (
    SELECT jsonb_build_object(
        'type', 'Feature',
        'id', iso,
        'geometry', ST_AsGeoJSON(ADM0simple)::jsonb,
        'properties', to_jsonb(row) - 'iso' - 'ADM0simple'
    ) AS feature
    FROM (
        SELECT *
        FROM countries
        WHERE iso IN (
            SELECT country_iso
            FROM Memberships
            WHERE Org_ID = %s
        )
    ) row
) features;
""",
[orgname]
            )
            return cur.fetchone()[0]

In [80]:
with open("two.geojson", "w") as fs:
    json.dump(get_org_geojson("EU"), fs)